In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import json
import csv
from pandas import DataFrame
import time
import gc
import h5py

from IPython.display import Image
from IPython.core.display import HTML

from scipy.sparse import csr_matrix
from sklearn import preprocessing

%matplotlib inline

C:\Users\anton\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import glob
import pickle

FEATURES_LOCATION = './features_resnet/'
F_CORE = 'cnn_features_'

def get_label_from_path(file):
    return file.split('\\')[-1].split('.')[0]

def load_data(mode):
    if(mode == 'test'):
        pickle_path = F_CORE + mode
        data = pickle.load(open(FEATURES_LOCATION + pickle_path + '.pkl', 'rb'))
        to_return = {}
        for key, value in list(data.items()):
            to_return[get_label_from_path(key)] = value.reshape(-1,)
        return to_return, None
    
    pickle_path = F_CORE + mode + '_'
    
    data = {}
    for i in range(1,129):
        data[i] = pickle.load(open(FEATURES_LOCATION + pickle_path + str(i) + '.pkl', 'rb'))
        
    X = []
    y = []
    for key, value in list(data.items()):
        the_class = key
        features = np.array(list(value.values()))
        for feature in features:
            y.append(the_class)
            X.append(feature)

    return np.array(X), np.array(y)

In [3]:
# Load data
X, y = load_data('train')
X_val, y_val = load_data('valid')

# Extract number of labels in the training data
num_labels = np.unique(y).shape[0]
num_features = X.shape[1]
num_trainobs = X.shape[0]

# Balance labels
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced', np.unique(y), y)

# Create one hot encoding for training and validation features
lb = preprocessing.LabelBinarizer()
lb.fit(y)
y = lb.transform(y)
y_val = lb.transform(y_val)

In [4]:
X_test, _ = load_data('test')
X_test_arr = np.array(list(X_test.values()))
X_test_arr = X_test_arr[:, np.newaxis, np.newaxis, :]

In [5]:
# Illustration that the code runs (The 79.41% val acc was achieved with more epochs)
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras import optimizers

model = Sequential()
model.add(Flatten(input_shape=(1,1,2048)))
model.add(Dropout(0.10))
model.add(Dense(units=1024, activation='tanh'))
model.add(Dropout(0.25))
model.add(Dense(units=128, activation='softmax'))

opt = keras.optimizers.Adam(lr=0.00025, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
callbacks= [keras.callbacks.ModelCheckpoint('checkpoint.h5', monitor='val_acc', verbose=2, save_best_only=True, mode='auto', period=1)]

model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X, y, epochs=30, batch_size=512, validation_data=(X_val, y_val), callbacks=callbacks, class_weight=class_weight)

Using TensorFlow backend.


Train on 192171 samples, validate on 6309 samples
Epoch 1/30
192171/192171 [==============================] - 16s 83us/step - loss: 1.3942 - acc: 0.6318 - val_loss: 1.0440 - val_acc: 0.6974

Epoch 00001: val_acc improved from -inf to 0.69742, saving model to checkpoint.h5
Epoch 2/30
192171/192171 [==============================] - 9s 45us/step - loss: 0.8912 - acc: 0.7394 - val_loss: 0.9344 - val_acc: 0.7261

Epoch 00002: val_acc improved from 0.69742 to 0.72611, saving model to checkpoint.h5
Epoch 3/30
192171/192171 [==============================] - 9s 45us/step - loss: 0.7798 - acc: 0.7690 - val_loss: 0.8838 - val_acc: 0.7396

Epoch 00003: val_acc improved from 0.72611 to 0.73958, saving model to checkpoint.h5
Epoch 4/30
192171/192171 [==============================] - 9s 46us/step - loss: 0.7154 - acc: 0.7853 - val_loss: 0.8523 - val_acc: 0.7483

Epoch 00004: val_acc improved from 0.73958 to 0.74830, saving model to checkpoint.h5
Epoch 5/30
192171/192171 [==========================

In [7]:
import keras
model_test = keras.models.load_model('ResNetTop_ValAcc7941.h5')
model_test.evaluate(X_val, y_val)

6309/6309 [==============================] - 1s 95us/step


[0.7973589453309476, 0.7941036614407675]

In [8]:
# Test predictions
y_pred_oh = model_test.predict(X_test_arr)
# Convert predictions from one-hot to actual labels and print csv
y_pred = lb.inverse_transform(y_pred_oh)

In [9]:
predictions = {}
for i, index in enumerate(X_test.keys()):
    predictions[int(index)] = y_pred[i]
    
from collections import Counter
counted = Counter(predictions.values())
most_common_class = counted.most_common()[0][0]

for index in range(1, 12801):
    if(index not in predictions.keys()):
        predictions[index] = most_common_class
        
ids = []
values = []
for key, value in predictions.items():
    ids.append(key)
    values.append(value)
    
out_dict = {}
out_dict['id'] = ids
out_dict['predicted'] = values

keys = sorted(out_dict.keys())
COL_WIDTH = 6
FMT = "%%-%ds" % COL_WIDTH

with open('predictions.csv', 'w') as csv:
    # Write keys    
    csv.write(','.join([k for k in keys]) + '\n')

    # Assume all values of dict are equal
    for i in range(len(out_dict[keys[0]])):
        csv.write(','.join([FMT % out_dict[k][i] for k in keys]) + '\n')

In [11]:
# Fine-tuning attempt
import keras
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Flatten, Input

# load pretrained ResNet50 model
resnet_model = ResNet50(weights='imagenet', 
                           include_top=False, 
                           input_shape=(224, 224, 3))

# Load prediction layer
model_clf = keras.models.load_model('ResNetTop_ValAcc7941.h5')

# Connect
last_flayer = resnet_model.output
out = model_clf(last_flayer)

# Collect
resnet_ft = Model(resnet_model.input, out)

# Fine-tune last layers
for i in range(0, len(resnet_ft.layers)):
    if i < 169 or i == 174:
        print("%i: Setting layer %s to non-trainable" % (i, str(resnet_ft.layers[i])))
        resnet_ft.layers[i].trainable=False
    else:
        print("%i: Keeping layer %s trainable" % (i, str(resnet_ft.layers[i])))

0: Setting layer <keras.engine.topology.InputLayer object at 0x00000154D1192DD8> to non-trainable
1: Setting layer <keras.layers.convolutional.Conv2D object at 0x00000154D1A5D2E8> to non-trainable
2: Setting layer <keras.layers.normalization.BatchNormalization object at 0x00000154D151BE10> to non-trainable
3: Setting layer <keras.layers.core.Activation object at 0x00000154D15B7F60> to non-trainable
4: Setting layer <keras.layers.pooling.MaxPooling2D object at 0x00000154D1669978> to non-trainable
5: Setting layer <keras.layers.convolutional.Conv2D object at 0x00000154D1BB1B00> to non-trainable
6: Setting layer <keras.layers.normalization.BatchNormalization object at 0x00000154CB8BF4E0> to non-trainable
7: Setting layer <keras.layers.core.Activation object at 0x00000154D1C07C88> to non-trainable
8: Setting layer <keras.layers.convolutional.Conv2D object at 0x00000154CB8E5C18> to non-trainable
9: Setting layer <keras.layers.normalization.BatchNormalization object at 0x00000154CB8D89E8> to

In [12]:
# Data generator
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

# Train
target_size = (224,224)
validation_dir = './valid'
train_dir = './train'

train_datagen = ImageDataGenerator(
        #rescale=1./255.
        #rotation_range=10,
        #width_shift_range=0.1,
        #height_shift_range=0.1,
        #horizontal_flip=True,
        preprocessing_function=preprocess_input
      )
 
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
 
train_batchsize = 128
val_batchsize = 128

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=target_size,
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=target_size,
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 192171 images belonging to 128 classes.
Found 6309 images belonging to 128 classes.


In [13]:
# Illustrate issue with model starting at 0 accuracy
optim = keras.optimizers.Adam(lr=(1e-5))

# Compile the model
resnet_ft.compile(loss='categorical_crossentropy',
            optimizer=optim,
            metrics=['acc'])

# Train the model
history = resnet_ft.fit_generator(
        train_generator,
        steps_per_epoch=200,
        epochs=300,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples/validation_generator.batch_size,
        verbose=1,
        callbacks=[keras.callbacks.ModelCheckpoint('ResnetFineTune.h5', monitor='val_loss', verbose=2, save_best_only=True, mode='auto', period=1)]
    )

Epoch 1/300
  6/200 [..............................] - ETA: 24:18 - loss: 13.2937 - acc: 0.0117

KeyboardInterrupt: 